In [19]:
iopub_data_rate_limit = 10e100

In [20]:
#importing libraries
import pandas as pd
import random
import numpy as np
from itertools import combinations

In [21]:
import csv
with open("hash_covid_tot.csv", encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    macro_list=[]
    lista=[]
    
    #hashtag lists normalization
    for row in csv_reader:
        macro_list.append(row[2])
    macro_list.pop(0)
    for k in macro_list:
        k = k.replace("[","")
        k = k.replace("]","")
        k = k.replace("'","")
        k = k.replace("\\","")
        k = k.replace("$","\%")
        k = k.split(",")
        k = [x for x in k if x is not '']
        k = sorted(k)
        if len(k)<2:    #single hashtag list removing
            del k
        else:
            elem = k
            new=[c.strip() for c in elem if c.strip()]
            comb = list(combinations(new,2))   #all possible hashtag combinations in a list
        lista.append(comb)   #put each combination in a list

In [22]:
len(lista)

15009

In [23]:
#creating the dataframe
nodo1 = []
nodo2 = []

for i in range(0,len(lista)):
    for j in range(0,len(lista[i])):
        nodo1.append(lista[i][j][0])
        nodo2.append(lista[i][j][1])
dataframe = pd.DataFrame({'nodo1':nodo1, 'nodo2':nodo2})
print(dataframe)

                nodo1                    nodo2
0        coronacrisis              coronavirus
1        coronacrisis  de wereld van personeel
2        coronacrisis           esther mallant
3        coronacrisis                 lockdown
4        coronacrisis               medewerker
...               ...                      ...
1200570        Techno                Godsendus
1200571         Verse                Godsendus
1200572      classics                   techno
1200573      classics                    house
1200574        techno                    house

[1200575 rows x 2 columns]


In [24]:
dataframe['nodo1'].unique()

array(['coronacrisis', 'coronavirus', 'de wereld van personeel', ...,
       'Steady Rock', 'Krushak', 'Verse'], dtype=object)

In [25]:
#computing weight for each pair of hashtag
data = dataframe.groupby(['nodo1','nodo2'],as_index=False).size().reset_index(name='peso').sort_values('peso',ascending=False)

In [26]:
data

,nodo1,nodo2,peso
389659,coronavirus,covid19,1066
389657,coronavirus,covid-19,694
395205,coronavirus,podcast,587
395627,coronavirus,quarantine,469
384810,corona,coronavirus,422
...,...,...,...
232764,Nerd,Pandemia,1
232763,Nerd,O Que é o Monomito,1
232762,Nerd,O Que é a Jornada do Herói,1
232761,Nerd,New,1


In [27]:
#preparing data for network creation
data = data[['nodo1','nodo2','peso']]
data['peso'] = data['peso'].astype(float)

In [28]:
data = data.rename(columns={"nodo1": "#nodo1", "nodo2": "#nodo2","peso":"#peso"})
data

,#nodo1,#nodo2,#peso
389659,coronavirus,covid19,1066.0
389657,coronavirus,covid-19,694.0
395205,coronavirus,podcast,587.0
395627,coronavirus,quarantine,469.0
384810,corona,coronavirus,422.0
...,...,...,...
232764,Nerd,Pandemia,1.0
232763,Nerd,O Que é o Monomito,1.0
232762,Nerd,O Que é a Jornada do Herói,1.0
232761,Nerd,New,1.0


In [30]:
data['#peso'].isnull().sum()

0

In [31]:
#removing search hashtag
str_to_drop = ['coronavirus', 'covid', 'Coronavirus', 'CORONAVIRUS','COVID','Covid']
dropped_nodo1 = data[~data['#nodo1'].str.contains('|'.join(str_to_drop))]
dropped_nodo2 = dropped_nodo1[~dropped_nodo1['#nodo2'].str.contains('|'.join(str_to_drop))]
dropped_nodo2

,#nodo1,#nodo2,#peso
178417,Hip Hop,rap,218.0
387454,corona,virus,193.0
372457,coach,coaching,187.0
372749,coaching,mentor,184.0
372542,coach,mentor,183.0
...,...,...,...
232764,Nerd,Pandemia,1.0
232763,Nerd,O Que é o Monomito,1.0
232762,Nerd,O Que é a Jornada do Herói,1.0
232761,Nerd,New,1.0


In [32]:
data.to_csv(r"C:\Users\clara\Desktop\NUOVOSNA\real_net.csv",index=False) #final csv creation